In [70]:
import numpy as np
import theano as th
from theano import tensor as T
from numpy import random as rng
 
class AutoEncoder(object):
    def __init__(self, X, hidden_size, activation_function,
                 output_function):
        #X is the data, an m x n numpy matrix
        #where rows correspond to datapoints
        #and columns correspond to features.
        assert type(X) is np.ndarray
        assert len(X.shape)==2
        self.X=X
        self.X=th.shared(name='X', value=np.asarray(self.X, 
                         dtype=th.config.floatX),borrow=True)
        #The config.floatX and borrow=True stuff is to get this to run
        #fast on the gpu. I recommend just doing this without thinking about
        #it until you understand the code as a whole, then learning more
        #about gpus and theano.
        self.n = X.shape[1]
        self.m = X.shape[0]
        #Hidden_size is the number of neurons in the hidden layer, an int.
        assert type(hidden_size) is int
        assert hidden_size > 0
        self.hidden_size=hidden_size
        initial_W = np.asarray(rng.uniform(
                 low=-4 * np.sqrt(6. / (self.hidden_size + self.n)),
                 high=4 * np.sqrt(6. / (self.hidden_size + self.n)),
                 size=(self.n, self.hidden_size)), dtype=th.config.floatX)
        self.W = th.shared(value=initial_W, name='W', borrow=True)
        self.b1 = th.shared(name='b1', value=np.zeros(shape=(self.hidden_size,),
                            dtype=th.config.floatX),borrow=True)
        self.b2 = th.shared(name='b2', value=np.zeros(shape=(self.n,),
                            dtype=th.config.floatX),borrow=True)
        self.activation_function=activation_function
        self.output_function=output_function
                     
    def train(self, n_epochs=10000, mini_batch_size=1, learning_rate=0.1):
        index = T.lscalar()
        x=T.matrix('x')
        params = [self.W, self.b1, self.b2]
        hidden = self.activation_function(T.dot(x, self.W)+self.b1)
        output = T.dot(hidden,T.transpose(self.W))+self.b2
        output = self.output_function(output)
         
        #Use cross-entropy loss.
        L = -T.sum(x*T.log(output) + (1-x)*T.log(1-output), axis=1)
        cost=L.mean()       
        updates=[]
         
        #Return gradient with respect to W, b1, b2.
        gparams = T.grad(cost,params)
         
        #Create a list of 2 tuples for updates.
        for param, gparam in zip(params, gparams):
            updates.append((param, param-learning_rate*gparam))
         
        #Train given a mini-batch of the data.
        train = th.function(inputs=[index], outputs=[cost], updates=updates,
                            givens={x:self.X[index:index+mini_batch_size,:]})
                             
 
        import time
        start_time = time.clock()
        for epoch in xrange(n_epochs):
            #print "Epoch:",epoch
            for row in xrange(0,self.m, mini_batch_size):
                train(row)
        end_time = time.clock()
        print "Average time per epoch=", (end_time-start_time)/n_epochs
        
     
    def get_weights(self):
        return [self.W.get_value(), self.b1.get_value(), self.b2.get_value()]         
               
    def get_hidden(self,data):
        x=T.dmatrix('x')
        hidden = self.activation_function(T.dot(x,self.W)+self.b1)
        transformed_data = th.function(inputs=[x], outputs=hidden)
        return transformed_data(data)  
    
    def get_output(self,data):
        y = T.dmatrix('y')
        output = T.dot(y,T.transpose(self.W))+self.b2
        reconstructed_data = th.function(inputs=[y], outputs=output)
        return reconstructed_data(data)


In [71]:
def m_test(data):
    X=data
    activation_function = T.nnet.sigmoid
    output_function=activation_function
    hidden_features = 9
    A = AutoEncoder(X, hidden_features, activation_function, output_function)
    A.train()
    W=np.transpose(A.get_weights()[0])
    ####################################################
    Y = A.get_hidden(X)
    Z = A.get_output(Y)
    ####################################################
    print X
    print "##############################bellow is a compressed data##################################"
    print Y
    print "##############################bellow is a reconstructed data##################################"
    print Z

In [72]:
import pandas as pd
from sklearn.cross_validation import train_test_split
#data = np.array([[1,2,5,3,4,5,6,9,8,7,4,5,6,9,8,7],[2,3,1,6,9,8,3,2,6,5,4,5,6,9,8,7],[1,2,5,8,9,6,4,7,5,3,4,5,6,9,8,7],[1,2,5,3,9,6,4,7,5,3,4,5,6,9,8,7],[1,2,5,1,9,6,4,7,5,3,1,2,5,3,9,6],[1,2,5,8,9,6,4,7,5,3,4,5,6,9,8,7],[1,2,9,8,9,6,4,7,5,3,6,4,7,5,3,4],[1,2,7,6,9,6,6,4,7,5,3,4,4,7,5,3],[1,2,5,8,9,6,4,7,5,3,4,5,6,9,8,7],[1,4,5,6,9,8,7,2,3,8,9,6,4,7,5,3]],dtype='f4')
data = pd.read_csv(open('/home/melvin/Documents/mellvin 15.04/Deep-Learning-Algoringms/wine.data'))
columns = ['class','alcohol','malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 
           'nonflavanoid phenols', 'proanthocyanins',  'color_intensity', 'hue', 'od_of_diluted_wines', 'proline']
data.columns = columns
class_coloumns = columns[1:]
X = data[class_coloumns].values
Y =  data[columns[0]]

X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.33, random_state=42)
X_train = np.array(X_train)
print data.shape

(177, 14)


In [73]:
#Scaling features to a range
from sklearn import preprocessing
X_train = np.array(X)
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_train = X_train_minmax

In [74]:
m_test(X_train)

Average time per epoch= 0.0185246533
[[ 0.57105263  0.2055336   0.4171123  ...,  0.46341463  0.78021978
   0.55064194]
 [ 0.56052632  0.3201581   0.70053476 ...,  0.44715447  0.6959707
   0.64693295]
 [ 0.87894737  0.23913043  0.60962567 ...,  0.30894309  0.7985348
   0.85734665]
 ..., 
 [ 0.58947368  0.69960474  0.48128342 ...,  0.08943089  0.10622711
   0.39728959]
 [ 0.56315789  0.36561265  0.54010695 ...,  0.09756098  0.12820513
   0.40085592]
 [ 0.81578947  0.66403162  0.73796791 ...,  0.10569106  0.12087912
   0.20114123]]
##############################bellow is a compressed data##################################
[[ 0.47696699  0.08290706  0.23256226 ...,  0.34196205  0.90223357
   0.61060427]
 [ 0.2638022   0.35229359  0.36509699 ...,  0.24597859  0.79024686
   0.91183548]
 [ 0.2279911   0.14728996  0.17314868 ...,  0.05112143  0.83694499
   0.78140579]
 ..., 
 [ 0.2223606   0.87323556  0.27617897 ...,  0.76199743  0.26348749
   0.16200281]
 [ 0.50852334  0.84322933  0.47654057 